In [3]:
import pandas as pd
# import matplotlib.pyplot as plt
import os
import scanpy
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader, Batch
from torch.utils.data import TensorDataset
from torch_geometric.transforms import RandomLinkSplit
from torch_geometric.utils import train_test_split_edges
from torch_geometric.nn import GATConv, GCNConv, SAGEConv, MessagePassing, Linear
import random
from scipy.io import mmread
import numpy as np
from sklearn.model_selection import train_test_split
from torcheval.metrics.functional import multiclass_f1_score
from sklearn.metrics import confusion_matrix
from collections import Counter

ModuleNotFoundError: No module named 'pandas'

In [2]:
import os
import argparse
import time
from datetime import datetime, date
import random

import numpy as np
from scipy.sparse import load_npz
from sklearn.metrics import roc_auc_score, f1_score, precision_recall_curve, auc
from scipy.stats import pearsonr
import pandas as pd

import torch
import torch_geometric
import torch.nn.functional as F
import torch.nn as nn


ModuleNotFoundError: No module named 'numpy'

In [2]:
hicmatrix  =pd.read_csv("../../chrX_1000000_hic.txt",header = None)
# hicmatrix_test = pd.read_csv("../../chrX_2000000_hic.txt",header = None)
epigenomic = pd.read_csv("../../train_epigemnomic_1000000.csv",header =  0).iloc[:,1:].to_numpy()
label = pd.read_csv("../../train_label.csv",header = 0).iloc[:,1:].to_numpy()


In [91]:
hicmatrix_train_np

array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       [  0.,   0.,   0., ...,   0.,   0.,   0.],
       ...,
       [  0.,   0.,   0., ..., 157., 269.,  24.],
       [  0.,   0.,   0., ..., 269., 130., 233.],
       [  0.,   0.,   0., ...,  24., 233., 134.]], dtype=float32)

In [42]:
# hicmatrix_train  =pd.read_csv("../../chrX_1000000_hic.txt",header = None)
# hicmatrix_test = pd.read_csv("../../chrX_2000000_hic.txt",header = None)
# epigenomic_test = pd.read_csv("../../test_epigemnomic_2000000.csv",header =  0).iloc[:,1:].to_numpy()
# label_test = pd.read_csv("../../test_label.csv",header = 0).iloc[:,1:].to_numpy()

In [4]:
epigenomic.shape

(2001, 24)

In [5]:
print(np.unique(label, return_counts=True)
)

(array([0, 1]), array([1950,   51]))


In [6]:
hicmatrix_np =hicmatrix.to_numpy(dtype=np.float32)
# hicmatrix_test_np = hicmatrix_test.to_numpy(dtype= np.float32)

In [7]:
# label = random_array

NameError: name 'random_array' is not defined

In [26]:
import torch
from torch_geometric.data import Data

edge_index = torch.tensor([[0, 1, 1, 2],
                           [1, 0, 2, 1]], dtype=torch.long)
x = torch.tensor([[-1], [0], [1]], dtype=torch.float)

data = Data(x=x, edge_index=edge_index)
data

Data(x=[3, 1], edge_index=[2, 4])

In [8]:
# From Whitney

def create_comp_graph(correlation, min_corr):
    '''Get a graph based on highly correlated features (feat) and given correlation (corr).'''
    s_l = [] # Gene 1 = Node 1
    d_l  = [] # Gene 2 = Node 2
    for i in range(0, correlation.shape[0]):
        for j in range(0, correlation.shape[1]):# Get edge and correlation
            if correlation[i][j] >= min_corr and i != j: # Ensure the features are at least given correlation (and it won't be a self correlated edge i. e. 1 --> 1)
                s_l.append(i)
                d_l.append(j)
            else:
                continue
    print(f"Number of edges: {len(s_l)}") # Prints # of edges so you can make sure it isn't empty/too large
    edge_index = np.array([s_l, d_l]) # Convert to array
    return edge_index # Return edge index

edge_index = create_comp_graph(hicmatrix_np, 10)
temp_edge_index = edge_index
# edge_index_test = create_comp_graph(hicmatrix_test_np,10)
# temp_edge_index_test = edge_index_test

Number of edges: 61946


In [24]:
edge_index

tensor([[  21,   22,   22,  ..., 2000, 2000, 2000],
        [  22,   21,   23,  ..., 1997, 1998, 1999]])

In [9]:
edge_index

array([[  21,   22,   22, ..., 2000, 2000, 2000],
       [  22,   21,   23, ..., 1997, 1998, 1999]])

In [10]:
def set_seed(seed: int = 12) -> None:
    '''This function allows us to set the seed for the notebook.'''
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ["PYTHONHASHSEED"] = str(seed)
    print(f"Random seed set as {seed}")

In [12]:
epsilon = 1
# training_exp_shuffled = training_exp_shuffled + epsilon
# training_exp_shuffled = np.log2(training_exp_shuffled)
# testing_exp_shuffled = testing_exp_shuffled + epsilon
# testing_exp_shuffled = np.log2(testing_exp_shuffled)

In [14]:
# edge_index_train = torch.tensor(edge_index_train, dtype=torch.int64)
# edge_index_test = torch.tensor(edge_index_test,dtype = torch.int64)
# train_data = []
# test_data = []
edge_index = torch.tensor(edge_index, dtype = torch.int64)
data = []
# Function to convert graph data to PyTorch Geometric Data object
def create_data_object(x, y, edge_index):
    x = torch.tensor(x, dtype=torch.float32)
    y = torch.tensor(y, dtype=torch.long)
    return Data(x=x, y=y, edge_index=edge_index)



/tmp/ipykernel_81557/2785758134.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  edge_index = torch.tensor(edge_index, dtype = torch.int64)


In [16]:
data = DataLoader(data, batch_size = 75)

/oscar/data/larschan/gcn_gps/lib/python3.8/site-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [27]:
import numpy as np
from scipy import sparse

In [39]:
from scipy.sparse import load_npz
import torch
import torch_geometric

In [47]:
###Define model inputs
hic_sparse_mat_file = sparse.csr_matrix(hicmatrix_np)
mat = (hic_sparse_mat_file)
# allNodes_hms = np.load(np_hmods_norm_all_file)
hms = epigenomic #only includes features, not node ids
X = torch.tensor(hms).float() 
allNodes = np.arange(2001).astype(int)
# geneNodes_labs = np.load(np_nodes_lab_genes_file)

# geneNodes = geneNodes_labs[:, -2].astype(int)
allLabs = label

targetNode_mask = torch.tensor(allNodes).long()


# geneLabs = geneNodes_labs[:, -1].astype(float)
# allLabs[geneNodes] = geneLabs
Y = torch.tensor(allLabs).long()

extract = torch_geometric.utils.from_scipy_sparse_matrix(mat)
data = torch_geometric.data.Data(edge_index = extract[0], edge_attr = extract[1], x = X, y = Y)
G = data


In [48]:
###Define convolutional and linear layer input/output sizes
graph_conv_layer_sizes = [24] + \
    [int(max(256, 256)) \
          for i in np.arange(1, 2, 1)] + [256]
        
lin_hidden_sizes = [graph_conv_layer_sizes[-1]] + \
    [int(max(256, 2)) \
          for i in np.arange(1, 3, 1)] + [2]

In [49]:
###Randomize node order and split into 70%/15%/15% training/validation/test sets
pred_idx_shuff = torch.randperm(targetNode_mask.shape[0])

fin_train = np.floor(0.7*pred_idx_shuff.shape[0]).astype(int)
fin_valid = np.floor(0.85*pred_idx_shuff.shape[0]).astype(int)
train_idx = pred_idx_shuff[:fin_train]
valid_idx = pred_idx_shuff[fin_train:fin_valid]
test_idx = pred_idx_shuff[fin_valid:]

train_gene_ID = targetNode_mask[train_idx].numpy()
valid_gene_ID = targetNode_mask[valid_idx].numpy()
test_gene_ID = targetNode_mask[test_idx].numpy()

In [51]:
class GCN_classification(nn.Module):

    def __init__(self, num_feat, num_graph_conv_layers, graph_conv_layer_sizes, num_lin_layers, lin_hidden_sizes, num_classes):
        '''
        Defines classification model class

        Parameters
        ----------
        num_feat [int]: Feature dimension (int)
        num_graph_conv_layers [int]: Number of graph convolutional layers (1, 2, or 3)
        graph_conv_layer_sizes [int]: Embedding size of graph convolutional layers 
        num_lin_layers [int]: Number of linear layers (1, 2, or 3)
        lin_hidden_sizes [int]: Embedding size of hidden linear layers
        num_classes [int]: Number of classes to be predicted(=2)

        Returns
        -------
        None.

        '''
        
        super(GCN_classification, self).__init__()

        self.num_graph_conv_layers = num_graph_conv_layers
        self.num_lin_layers = num_lin_layers
        self.dropout_value = 0.5

        if self.num_graph_conv_layers == 1:
            self.conv1 = SAGEConvCat(graph_conv_layer_sizes[0], graph_conv_layer_sizes[1])
        elif self.num_graph_conv_layers == 2:
            self.conv1 = SAGEConvCat(graph_conv_layer_sizes[0], graph_conv_layer_sizes[1])
            self.conv2 = SAGEConvCat(graph_conv_layer_sizes[1], graph_conv_layer_sizes[2])
        elif self.num_graph_conv_layers == 3:
            self.conv1 = SAGEConvCat(graph_conv_layer_sizes[0], graph_conv_layer_sizes[1])
            self.conv2 = SAGEConvCat(graph_conv_layer_sizes[1], graph_conv_layer_sizes[2])
            self.conv3 = SAGEConvCat(graph_conv_layer_sizes[2], graph_conv_layer_sizes[3])
        
        if self.num_lin_layers == 1:
            self.lin1 = nn.Linear(lin_hidden_sizes[0], lin_hidden_sizes[1])
        elif self.num_lin_layers == 2:
            self.lin1 = nn.Linear(lin_hidden_sizes[0], lin_hidden_sizes[1])
            self.lin2 = nn.Linear(lin_hidden_sizes[1], lin_hidden_sizes[2])
        elif self.num_lin_layers == 3:
            self.lin1 = nn.Linear(lin_hidden_sizes[0], lin_hidden_sizes[1])
            self.lin2 = nn.Linear(lin_hidden_sizes[1], lin_hidden_sizes[2])
            self.lin3 = nn.Linear(lin_hidden_sizes[2], lin_hidden_sizes[3])
            
        self.loss_calc = nn.CrossEntropyLoss()
        self.torch_softmax = nn.Softmax(dim=1)
        
        
    def forward(self, x, edge_index, train_status=False):
        '''
        Forward function.
        
        Parameters
        ----------
        x [tensor]: Node features
        edge_index [tensor]: Subgraph mask
        train_status [bool]: optional, set to True for dropout

        Returns
        -------
        scores [tensor]: Pre-normalized class scores

        '''

        ### Graph convolution module
        if self.num_graph_conv_layers == 1:
            h = self.conv1(x, edge_index)
            h = torch.relu(h)
        elif self.num_graph_conv_layers == 2:
            h = self.conv1(x, edge_index)
            h = torch.relu(h)
            h = self.conv2(h, edge_index)
            h = torch.relu(h)
        elif self.num_graph_conv_layers == 3:
            h = self.conv1(x, edge_index)
            h = torch.relu(h)
            h = self.conv2(h, edge_index)
            h = torch.relu(h)
            h = self.conv3(h, edge_index)
            h = torch.relu(h)
            
        h = F.dropout(h, p = self.dropout_value, training=train_status)

        ### Linear module
        if self.num_lin_layers == 1:
            scores = self.lin1(h)
        elif self.num_lin_layers == 2:
            scores = self.lin1(h)
            scores = torch.relu(scores)
            scores = self.lin2(scores)
        elif self.num_lin_layers == 3:
            scores = self.lin1(h)
            scores = torch.relu(scores)
            scores = self.lin2(scores)
            scores = torch.relu(scores)
            scores = self.lin3(scores)
        
        return scores
    
    
    def loss(self, scores, labels):
        '''
        Calculates cross-entropy loss
        
        Parameters
        ----------
        scores [tensor]: Pre-normalized class scores from forward function
        labels [tensor]: Class labels for nodes

        Returns
        -------
        xent_loss [tensor]: Cross-entropy loss

        '''

        xent_loss = self.loss_calc(scores, labels)

        return xent_loss
    
    
    def calc_softmax_pred(self, scores):
        '''
        Calculates softmax scores and predicted classes

        Parameters
        ----------
        scores [tensor]: Pre-normalized class scores

        Returns
        -------
        softmax [tensor]: Probability for each class
        predicted [tensor]: Predicted class

        '''
        
        softmax = self.torch_softmax(scores)
        
        predicted = torch.argmax(softmax, 1)
        
        return softmax, predicted

In [ ]:
from torch_sparse import SparseTensor, matmul


In [ ]:
import torch
from typing import Union, Tuple
from torch_geometric.typing import OptPairTensor, Adj, Size

from torch import Tensor
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn.conv import MessagePassing

In [61]:
class SAGEConvCat(MessagePassing):
    """
    *Note: Source function taken from PyTorch Geometric and modified such that
    embeddings are first concatenated and then reduced to out_channel size as
    per the original GraphSAGE paper.
    
    The GraphSAGE operator from the `"Inductive Representation Learning on
    Large Graphs" <https://arxiv.org/abs/1706.02216>`_ paper
    .. math::
        \mathbf{x}^{\prime}_i = \mathbf{W}_1 \mathbf{x}_i + \mathbf{W_2} \cdot
        \mathrm{mean}_{j \in \mathcal{N(i)}} \mathbf{x}_j
    Args:
        in_channels (int or tuple): Size of each input sample. A tuple
            corresponds to the sizes of source and target dimensionalities.
        out_channels (int): Size of each output sample.
        normalize (bool, optional): If set to :obj:`True`, output features
            will be :math:`\ell_2`-normalized, *i.e.*,
            :math:`\frac{\mathbf{x}^{\prime}_i}
            {\| \mathbf{x}^{\prime}_i \|_2}`.
            (default: :obj:`False`)
        bias (bool, optional): If set to :obj:`False`, the layer will not learn
            an additive bias. (default: :obj:`True`)
        **kwargs (optional): Additional arguments of
            :class:`torch_geometric.nn.conv.MessagePassing`.

    PyTorch Geometric citation:
    @inproceedings{Fey/Lenssen/2019,
      title={Fast Graph Representation Learning with {PyTorch Geometric}},
      author={Fey, Matthias and Lenssen, Jan E.},
      booktitle={ICLR Workshop on Representation Learning on Graphs and Manifolds},
      year={2019},
    }
    """
    
    def __init__(self, in_channels: Union[int, Tuple[int, int]],
                 out_channels: int, normalize: bool = False,
                 bias: bool = True, **kwargs):  # yapf: disable
        super(SAGEConvCat, self).__init__(aggr='mean', **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.normalize = normalize

        if isinstance(in_channels, int):
            in_channels = (in_channels, in_channels)

        self.lin_l = Linear(in_channels[0]*2, out_channels, bias=bias)
        
        self.reset_parameters()

    def reset_parameters(self):
        self.lin_l.reset_parameters()

    def forward(self, x: Union[Tensor, OptPairTensor], edge_index: Adj,
                size: Size = None) -> Tensor:
        
        out = self.propagate(edge_index, x=x, size=size)

        ### Concatenation
        out = torch.cat([x, out], dim=-1)
        out = self.lin_l(out)

        if self.normalize:
            out = F.normalize(out, p=2., dim=-1)

        return out

    def message(self, x_j: Tensor) -> Tensor:
        return x_j

    def message_and_aggregate(self, adj_t: SparseTensor,
                              x: OptPairTensor) -> Tensor:
        adj_t = adj_t.set_value(None, layout=None)
        return matmul(adj_t, x[0], reduce=self.aggr)

    def __repr__(self):
        return '{}({}, {})'.format(self.__class__.__name__, self.in_channels,
                                   self.out_channels)

NameError: name 'SparseTensor' is not defined

In [57]:
model = GCN_classification(24, 3, graph_conv_layer_sizes, 3, lin_hidden_sizes, 2)


NameError: name 'SAGEConvCat' is not defined

In [19]:
from torch_geometric.nn import global_mean_pool, global_max_pool
from torch_geometric.graphgym import init_weights

class GCN(torch.nn.Module):
    def __init__(self, hidden_dim1, num_labels):
        super(GCN, self).__init__()
        self.conv1 = SAGEConv(1, hidden_dim1)
        self.init_weights(self.conv1)
        # self.conv2 = SAGEConv(hidden_dim1, hidden_dim2)
        # self.init_weights(self.conv2)
        self.conv3 = SAGEConv(hidden_dim1, num_labels)
        self.init_weights(self.conv3)
        # self.conv4 = SAGEConv(hidden_dim3, num_labels)
        # self.init_weights(self.conv4)
        self.lin = Linear(num_labels, num_labels)
    def forward(self, x, edge_index, batch=None):
        print(edge_index)
        x = F.leaky_relu(self.conv1(x, edge_index))
       # x = F.leaky_relu(self.conv2(x, edge_index))
        x = F.leaky_relu(self.conv3(x, edge_index))
        # x = F.leaky_relu(self.conv4(x, edge_index))
        x = global_mean_pool(x, batch) 
        x = self.lin(x)
        x = F.log_softmax(x, dim=edge_index)
        return x

    def init_weights(self, layer):
        if hasattr(layer, 'weight') and layer.weight is not None:
            xavier_uniform_(layer.weight.data)


/oscar/data/larschan/gcn_gps/lib/python3.8/site-packages/torch_geometric/graphgym/config.py:19: UserWarning: Could not define global config object. Please install 'yacs' via 'pip install yacs' in order to use GraphGym
  warnings.warn("Could not define global config object. Please install "
/oscar/data/larschan/gcn_gps/lib/python3.8/site-packages/torch_geometric/graphgym/imports.py:14: UserWarning: Please install 'pytorch_lightning' via  'pip install pytorch_lightning' in order to use GraphGym
  warnings.warn("Please install 'pytorch_lightning' via  "


In [20]:
from sklearn.metrics import roc_auc_score
import tqdm


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
num_labels=4

model = GCN(64, num_labels).to(device)
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion =  nn.CrossEntropyLoss() # change loss for binary classification vs multiclass

def accuracy(predictions, targets):
    # Assuming predictions and targets are tensors
    _, predicted_classes = predictions.max(dim=1)
    correct = (predicted_classes == targets).sum().item()
    accuracy = correct / targets.size(0)
    return accuracy

def train(train_loader):
    model.train()
    avgLoss = 0
    avgAcc = 0
    for data in tqdm.tqdm(train_loader):
        optimizer.zero_grad()
        data.x = torch.reshape(data.x, (data.x.shape[0], 1)).to(torch.float32).to(device)
        # data.x = data.x.view(-1, 1).to(torch.float32)
        out = model(data.x, data.edge_index, data.batch)
        #target = torch.reshape(data.y, (100,2)) # CHANGE DIMENSIONS AS YOU CHANGE BATCH SIZE
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        avgLoss += loss
        avgAcc += accuracy(out, data.y)
    return (avgLoss / len(train_loader)), (avgAcc / len(train_loader))
def test(loader):
    model.eval()
    correct = 0
    for data in loader: 
            # data.x = data.x.view(-1, 1).to(torch.float32)
        data.x = torch.reshape(data.x, (data.x.shape[0], 1)).to(torch.float32).to(device)
        out = model(data.x, data.edge_index, data.batch)
        #target = torch.reshape(data.y, (100,2))
        pred = torch.argmax(out, dim=1)
        correct += int((pred == data.y).sum())
    return correct / len(loader.dataset)

GCN(
  (conv1): SAGEConv(1, 64, aggr=mean)
  (conv3): SAGEConv(64, 4, aggr=mean)
  (lin): Linear(4, 4, bias=True)
)


In [23]:
for epoch in range(1,21):
    loss, train_acc = train(data)
#     test_acc = test(test_loader)
    print(f'Epoch: {epoch}, Train: {train_acc}, Test Accuracy: {test_acc}')

  0%|          | 0/27 [00:00<?, ?it/s]


tensor([[  21,   22,   22,  ..., 3776, 3776, 3776],
        [  22,   21,   23,  ..., 3773, 3774, 3775]])


IndexError: Encountered an index error. Please ensure that all indices in 'edge_index' point to valid indices in the interval [0, 1799] (got interval [21, 3776])

In [107]:
train_loader